In [1]:
import pyfdb 
import emohawk

<div class="alert alert-block alert-warning">
To run this scirpt FDB access is needed and the <b>FDB_HOME</b> environment variable has to be set correctly.
</div>

In [ ]:
request = {
    'class': 'od',
    'expver': '0001',
    'stream': 'oper',
    'date': '20221207',
    'time': '0000',
    'domain': 'g',
    'type': 'an',
    'levtype': 'sfc',
    'step': 0,
    'param': [151, 167]
}

# Must be set correctly
%env FDB_HOME=

fdb = pyfdb.FDB()

#### Using iteration only

In [3]:
r = fdb.retrieve(request)

fs = emohawk.load_from("stream-iter", r)
type(fs)

fieldlist.sources.stream_iter.GribStreamIteratorSource

Nothing is read at this moment.

We can only use fs for iteration. Fields crerated in the iteration get deleted when going out of scope:

In [4]:
for f in fs:
    print(f"type(f)={type(f)}")
    print(f"  path={f.path} offset={f.offset}")
    print(f"  param={f['param']}")
    print(f"  shape={f.values.shape} mean={f.values.mean()}")

type(f)=<class 'fieldlist.readers.grib.memory.GribFieldInMemory'>
  path=None offset=None
  param=msl
  shape=(6599680,) mean=101181.03346484223
type(f)=<class 'fieldlist.readers.grib.memory.GribFieldInMemory'>
  path=None offset=None
  param=2t
  shape=(6599680,) mean=287.5265971353204


Once the iteration is completed, there is nothing left in fs.

In [5]:
for f in fs:
    print(f"type(f)={type(f)}")

#### Storing all the fields in memory

In [6]:
r = fdb.retrieve(request)

fs = emohawk.load_from("stream", r)
type(fs)

fieldlist.sources.stream.GribStream

Nothing is read at this moment:

In [7]:
print(f"stored fields count={len(fs._reader._fields)}")

stored fields count=0


If we call any function on the fieldlist it reads the messages into memory

In [8]:
len(fs)

2

In [9]:
print(f"stored fields count={len(fs._reader._fields)}")

stored fields count=2


In [10]:
fs.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,msl,surface,0,20221207,0,0,an,0,reduced_gg
1,ecmf,2t,surface,0,20221207,0,0,an,0,reduced_gg


In [11]:
a = fs.sel(param="2t")
print(type(a))

<class 'fieldlist.readers.grib.index.MaskFieldSet'>


In [12]:
a.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,2t,surface,0,20221207,0,0,an,0,reduced_gg


In [ ]:
fs.to_xarray()